In [1]:
from __future__ import print_function
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding, BatchNormalization
from keras.layers import LSTM, SimpleRNN, GRU
from keras.datasets import imdb
from keras.utils.np_utils import to_categorical
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import Normalizer
import h5py
from keras import callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn.metrics import classification_report
from keras import optimizers
import datetime
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score


import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
set_session(tf.Session(config=config))

# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_

# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(
        target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    # Regression
    return df[result].values.astype(np.float32), df[[target]].values.astype(np.float32)


traindata = pd.read_csv('KDDTrain+.csv', header=None)
testdata = pd.read_csv('KDDTest+.csv', header=None)


columns = [
    'duration',
    'protocol_type',
    'service',
    'flag',
    'src_bytes',
    'dst_bytes',
    'land',
    'wrong_fragment',
    'urgent',
    'hot',
    'num_failed_logins',
    'logged_in',
    'num_compromised',
    'root_shell',
    'su_attempted',
    'num_root',
    'num_file_creations',
    'num_shells',
    'num_access_files',
    'num_outbound_cmds',
    'is_host_login',
    'is_guest_login',
    'count',
    'srv_count',
    'serror_rate',
    'srv_serror_rate',
    'rerror_rate',
    'srv_rerror_rate',
    'same_srv_rate',
    'diff_srv_rate',
    'srv_diff_host_rate',
    'dst_host_count',
    'dst_host_srv_count',
    'dst_host_same_srv_rate',
    'dst_host_diff_srv_rate',
    'dst_host_same_src_port_rate',
    'dst_host_srv_diff_host_rate',
    'dst_host_serror_rate',
    'dst_host_srv_serror_rate',
    'dst_host_rerror_rate',
    'dst_host_srv_rerror_rate',
    'class',
    'difficulty_level'
]
traindata.columns=columns
testdata.columns=columns

del traindata["difficulty_level"]
del testdata["difficulty_level"]
traindata.info()
testdata.info()
trainlen=traindata.shape[0]

Using TensorFlow backend.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125973 entries, 0 to 125972
Data columns (total 42 columns):
duration                       125973 non-null int64
protocol_type                  125973 non-null object
service                        125973 non-null object
flag                           125973 non-null object
src_bytes                      125973 non-null int64
dst_bytes                      125973 non-null int64
land                           125973 non-null int64
wrong_fragment                 125973 non-null int64
urgent                         125973 non-null int64
hot                            125973 non-null int64
num_failed_logins              125973 non-null int64
logged_in                      125973 non-null int64
num_compromised                125973 non-null int64
root_shell                     125973 non-null int64
su_attempted                   125973 non-null int64
num_root                       125973 non-null int64
num_file_creations             125973 

In [2]:
alldata=pd.concat([traindata, testdata], axis=0, join='outer',ignore_index=True)
print(alldata)
#print(alldata['protocol_type'])
class_names=alldata['class'].unique()
print(class_names)

mask = alldata['class'] != 'normal' 
alldata.loc[mask, 'class'] = 'attack'
alldata['class'].value_counts()

#alldata.head()

        duration protocol_type     service  flag  src_bytes  dst_bytes  land  \
0              0           tcp    ftp_data    SF        491          0     0   
1              0           udp       other    SF        146          0     0   
2              0           tcp     private    S0          0          0     0   
3              0           tcp        http    SF        232       8153     0   
4              0           tcp        http    SF        199        420     0   
5              0           tcp     private   REJ          0          0     0   
6              0           tcp     private    S0          0          0     0   
7              0           tcp     private    S0          0          0     0   
8              0           tcp  remote_job    S0          0          0     0   
9              0           tcp     private    S0          0          0     0   
10             0           tcp     private   REJ          0          0     0   
11             0           tcp     priva

normal    77053
attack    71463
Name: class, dtype: int64

In [3]:

#traindata.info()
#traindata.describe()
protocol_type_dummy=pd.get_dummies(alldata['protocol_type'])
alldata=pd.concat([alldata,protocol_type_dummy],axis=1)
del alldata['protocol_type']


service_dummy=pd.get_dummies(alldata["service"])
alldata=pd.concat([alldata,service_dummy],axis=1)
del alldata["service"]

flag_dummy=pd.get_dummies(alldata["flag"])
alldata=pd.concat([alldata,flag_dummy],axis=1)
del alldata["flag"]
print(alldata.head())

#print(protocol_type_dummy)

encode_text_index(alldata, 'class')
# Break into X (predictors) & y (prediction)
x, y = to_xy(alldata,'class')

traindata=alldata.ix[0:trainlen-1,:]
testdata=alldata.ix[trainlen:alldata.shape[0],:]

   duration  src_bytes  dst_bytes  land  wrong_fragment  urgent  hot  \
0         0        491          0     0               0       0    0   
1         0        146          0     0               0       0    0   
2         0          0          0     0               0       0    0   
3         0        232       8153     0               0       0    0   
4         0        199        420     0               0       0    0   

   num_failed_logins  logged_in  num_compromised  ...  REJ  RSTO  RSTOS0  \
0                  0          0                0  ...    0     0       0   
1                  0          0                0  ...    0     0       0   
2                  0          0                0  ...    0     0       0   
3                  0          1                0  ...    0     0       0   
4                  0          1                0  ...    0     0       0   

   RSTR  S0  S1  S2  S3  SF  SH  
0     0   0   0   0   0   1   0  
1     0   0   0   0   0   1   0  
2     0 

/home/dsu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/home/dsu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [4]:
#X = traindata.iloc[:,1:42]
#Y = traindata.iloc[:,0]
#C = testdata.iloc[:,0]
#T = testdata.iloc[:,1:42]

X = traindata.drop('class', axis=1)
Y = traindata[['class']]
T = testdata.drop('class',axis=1)
C = testdata[['class']]

scaler = Normalizer().fit(X)
trainX = scaler.transform(X)
# summarize transformed data
np.set_printoptions(precision=3)
#print(trainX[0:5,:])

scaler = Normalizer().fit(T)
testT = scaler.transform(T)
# summarize transformed data
np.set_printoptions(precision=3)
#print(testT[0:5,:])

print("X info")
X.info()
print("Y info")
Y.info()
print("T info")
T.info()
print("C info")
C.info()
y_train = np.array(Y)
y_test = np.array(C)


# reshape input to be [samples, time steps, features]
X_train = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
X_test = np.reshape(testT, (testT.shape[0], 1, testT.shape[1]))


print(X_train.shape)
print(X_test.shape)
print(X.columns)
print(T.columns)


X info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125973 entries, 0 to 125972
Columns: 122 entries, duration to SH
dtypes: float64(15), int64(23), uint8(84)
memory usage: 46.6 MB
Y info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125973 entries, 0 to 125972
Data columns (total 1 columns):
class    125973 non-null int64
dtypes: int64(1)
memory usage: 984.2 KB
T info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22543 entries, 125973 to 148515
Columns: 122 entries, duration to SH
dtypes: float64(15), int64(23), uint8(84)
memory usage: 8.3 MB
C info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22543 entries, 125973 to 148515
Data columns (total 1 columns):
class    22543 non-null int64
dtypes: int64(1)
memory usage: 176.2 KB
(125973, 1, 122)
(22543, 1, 122)
Index(['duration', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment',
       'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised',
       ...
       'REJ', 'RSTO', 'RSTOS0', 'RSTR', 'S0', 'S1', 'S

# all 122 features

In [5]:
print("TIME BEGIN:",datetime.datetime.now())
batch_size = 32
n_neurons=80

# 1. define the network
model = Sequential()
#model.add(LSTM(4,input_dim=(1,122)))  # try using a GRU instead, for fun
#model.add(LSTM(4,input_shape=(1,122)))
model.add(LSTM(n_neurons, input_shape=(1,122)))
model.add(Dropout(0.1))
model.add(Dense(1))
model.add(Activation('sigmoid'))
#print(model.get_config())


#model = Sequential([
#    LSTM(4,input_shape=(1,122)),
#    Dense(len(class_names), activation='softmax')
#])

#model.compile(
#    optimizer='adam',
#    loss='sparse_categorical_crossentropy',
#    metrics=['accuracy'])


# try using different optimizers and different optimizer configs
sgd = optimizers.SGD(lr=0.5)
model.compile(loss='binary_crossentropy',optimizer=sgd,metrics=['accuracy'])
csv_logger = CSVLogger('all122featuresetlog.csv',separator=',', append=False)
#checkpointer = callbacks.ModelCheckpoint(filepath="kddresults/lstm1layer/checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='val_acc',mode='max')
#csv_logger = CSVLogger('training_set_iranalysis.csv',separator=',', append=False)
#model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=1000, validation_data=(X_test, y_test),callbacks=[checkpointer,csv_logger])
model.fit(X_train, y_train, batch_size=batch_size, epochs=45, validation_data=(X_test, y_test))

#model.save("kddresults/lstm1layer/fullmodel/lstm1layer_model.hdf5")
# Save the model
model.save('all122featuresModel.h5')

loss, accuracy = model.evaluate(X_test, y_test)
print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy*100))
y_pred = model.predict_classes(X_test)
print(classification_report(y_test, y_pred))
#np.savetxt('kddresults/lstm1layer/lstm1predicted.txt', y_pred, fmt='%01d')
print("TIME END:",datetime.datetime.now())

W1001 15:05:35.893579 140236905699136 deprecation_wrapper.py:119] From /home/dsu/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:68: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1001 15:05:35.896083 140236905699136 deprecation_wrapper.py:119] From /home/dsu/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:504: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1001 15:05:35.899833 140236905699136 deprecation_wrapper.py:119] From /home/dsu/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3828: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1001 15:05:36.072204 140236905699136 deprecation_wrapper.py:119] From /home/dsu/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:126: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_defa

TIME BEGIN: 2019-10-01 15:05:35.893243


W1001 15:05:36.154644 140236905699136 deprecation_wrapper.py:119] From /home/dsu/anaconda3/lib/python3.7/site-packages/keras/optimizers.py:744: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1001 15:05:36.170573 140236905699136 deprecation_wrapper.py:119] From /home/dsu/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3066: The name tf.log is deprecated. Please use tf.math.log instead.

W1001 15:05:36.193103 140236905699136 deprecation.py:323] From /home/dsu/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 125973 samples, validate on 22543 samples
Epoch 1/45
125973/125973 [==============================] - 53s 418us/step - loss: 0.1925 - acc: 0.9199 - val_loss: 1.0058 - val_acc: 0.7497
Epoch 2/45
125973/125973 [==============================] - 49s 392us/step - loss: 0.1411 - acc: 0.9491 - val_loss: 1.1199 - val_acc: 0.7407
Epoch 3/45
125973/125973 [==============================] - 50s 393us/step - loss: 0.1228 - acc: 0.9582 - val_loss: 1.1274 - val_acc: 0.7467
Epoch 4/45
125973/125973 [==============================] - 49s 390us/step - loss: 0.1156 - acc: 0.9607 - val_loss: 1.1018 - val_acc: 0.7661
Epoch 5/45
125973/125973 [==============================] - 50s 397us/step - loss: 0.1094 - acc: 0.9621 - val_loss: 1.0644 - val_acc: 0.7690
Epoch 6/45
125973/125973 [==============================] - 49s 391us/step - loss: 0.1051 - acc: 0.9633 - val_loss: 1.1171 - val_acc: 0.7564
Epoch 7/45
125973/125973 [==============================] - 49s 388us/step - loss: 0.1019 - acc: 0.9641

In [6]:
metrics.f1_score(y_test, y_pred, labels=np.unique(y_pred))

0.796867724867725

In [7]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test,y_pred).ravel()
print(tn)
print(fp)
print(fn)
print(tp)
confusion_matrix(y_test,y_pred)

8331
4502
297
9413


array([[8331, 4502],
       [ 297, 9413]])

In [8]:
from sklearn.metrics import f1_score
metrics.f1_score(y_test, y_pred, labels=np.unique(y_pred))


0.796867724867725

# 3 feature set

In [6]:
alldata.head()
#col_list=['dst_host_diff_srv_rate','dst_host_serror_rate','serror_rate','dst_host_same_src_port_rate','S0','dst_host_srv_serror_rate','class']
#col_list=['class','ecr_i','src_bytes','http','dst_bytes','count','private','dst_host_srv_count','dst_host_same_srv_rate','dst_host_same_src_port_rate','hot','dst_host_srv_serror_rate','is_guest_login','icmp','REJ','dst_host_serror_rate','dst_host_srv_diff_host_rate','tcp','urp_i','dst_host_rerror_rate','finger']
#col_list=['class','ecr_i','src_bytes','http','dst_bytes','count','private','dst_host_srv_count','dst_host_same_srv_rate','dst_host_same_src_port_rate','hot']
#col_list=['class','ecr_i','src_bytes','http','dst_bytes','count']


#10 feature set
#col_list=['class','src_bytes', 'dst_bytes','hot','count','dst_host_srv_count','dst_host_same_srv_rate','dst_host_same_src_port_rate','ecr_i','http','private']

#8 feature set
#col_list=['class','src_bytes', 'dst_bytes','hot','count','dst_host_same_srv_rate','ecr_i','http','private']

#6 feature set
#col_list=['class','src_bytes', 'dst_bytes','hot','count','ecr_i','http']

#4 feature set
#col_list=['class','src_bytes', 'dst_bytes','ecr_i','http']

#3 feature set
col_list=['class','src_bytes', 'ecr_i','http']

#col_list=['class','ecr_i','http','src_bytes','hot','dst_bytes','dst_host_same_srv_rate']
alldata2=alldata[col_list]
traindata=alldata2.ix[0:trainlen-1,:]
testdata=alldata2.ix[trainlen:alldata2.shape[0],:]


#X = traindata.iloc[:,1:42]
#Y = traindata.iloc[:,0]
#C = testdata.iloc[:,0]
#T = testdata.iloc[:,1:42]

X = traindata.drop('class', axis=1)
Y = traindata[['class']]
T = testdata.drop('class',axis=1)
C = testdata[['class']]

scaler = Normalizer().fit(X)
trainX = scaler.transform(X)
# summarize transformed data
np.set_printoptions(precision=3)
#print(trainX[0:5,:])

scaler = Normalizer().fit(T)
testT = scaler.transform(T)
# summarize transformed data
np.set_printoptions(precision=3)
#print(testT[0:5,:])

print("X info")
X.info()
print("Y info")
Y.info()
print("T info")
T.info()
print("C info")
C.info()
y_train = np.array(Y)
y_test = np.array(C)


# reshape input to be [samples, time steps, features]
X_train = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
X_test = np.reshape(testT, (testT.shape[0], 1, testT.shape[1]))


print(X_train.shape)
print(X_test.shape)
print(X.columns)
print(T.columns)

batch_size = 32

# 1. define the network
#model = Sequential()
#model.add(LSTM(4,input_dim=(1,122)))  # try using a GRU instead, for fun



# configure network
print("TIME BEGIN:",datetime.datetime.now())
n_epoch = 100
n_neurons = 80
# design network
model = Sequential()





model.add(LSTM(n_neurons,input_shape=(1,3)))
model.add(Dropout(0.1))
model.add(Dense(1))
model.add(Activation('sigmoid'))
#print(model.get_config())


#model = Sequential([
#    LSTM(4,input_shape=(1,122)),
#    Dense(len(class_names), activation='softmax')
#])

#model.compile(
#    optimizer='adam',
#    loss='sparse_categorical_crossentropy',
#    metrics=['accuracy'])


# try using different optimizers and different optimizer configs
sgd = optimizers.SGD(lr=0.5)
model.compile(loss='binary_crossentropy',optimizer=sgd,metrics=['accuracy'])
#model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

#checkpointer = callbacks.ModelCheckpoint(filepath="kddresults/lstm1layer/checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='val_acc',mode='max')
csv_logger = CSVLogger('3featuresetlog.csv',separator=',', append=False)
#model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=1000, validation_data=(X_test, y_test),callbacks=[checkpointer,csv_logger])
model.fit(X_train, y_train, batch_size=batch_size, epochs=45, validation_data=(X_test, y_test),callbacks=[csv_logger])

#model.save("kddresults/lstm1layer/fullmodel/lstm1layer_model.hdf5")
# Save the model
model.save('3featureModel.h5')

loss, accuracy = model.evaluate(X_test, y_test)
print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy*100))
y_pred = model.predict_classes(X_test)
print(classification_report(y_test, y_pred))
#np.savetxt('kddresults/lstm1layer/lstm1predicted.txt', y_pred, fmt='%01d')

print("TIME END:",datetime.datetime.now())






tn, fp, fn, tp = confusion_matrix(y_test,y_pred).ravel()
print(tn)
print(fp)
print(fn)
print(tp)
confusion_matrix(y_test,y_pred)

/home/dsu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/home/dsu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


X info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125973 entries, 0 to 125972
Data columns (total 3 columns):
src_bytes    125973 non-null int64
ecr_i        125973 non-null uint8
http         125973 non-null uint8
dtypes: int64(1), uint8(2)
memory usage: 1.2 MB
Y info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125973 entries, 0 to 125972
Data columns (total 1 columns):
class    125973 non-null int64
dtypes: int64(1)
memory usage: 984.2 KB
T info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22543 entries, 125973 to 148515
Data columns (total 3 columns):
src_bytes    22543 non-null int64
ecr_i        22543 non-null uint8
http         22543 non-null uint8
dtypes: int64(1), uint8(2)
memory usage: 220.2 KB
C info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22543 entries, 125973 to 148515
Data columns (total 1 columns):
class    22543 non-null int64
dtypes: int64(1)
memory usage: 176.2 KB
(125973, 1, 3)
(22543, 1, 3)
Index(['src_bytes', 'ecr_i', 'http'], dtype='obje

array([[7135, 5698],
       [ 138, 9572]])

In [7]:
metrics.f1_score(y_test, y_pred, labels=np.unique(y_pred))

0.7663730984787831

# 4 feature set

In [5]:
alldata.head()
#col_list=['dst_host_diff_srv_rate','dst_host_serror_rate','serror_rate','dst_host_same_src_port_rate','S0','dst_host_srv_serror_rate','class']
#col_list=['class','ecr_i','src_bytes','http','dst_bytes','count','private','dst_host_srv_count','dst_host_same_srv_rate','dst_host_same_src_port_rate','hot','dst_host_srv_serror_rate','is_guest_login','icmp','REJ','dst_host_serror_rate','dst_host_srv_diff_host_rate','tcp','urp_i','dst_host_rerror_rate','finger']
#col_list=['class','ecr_i','src_bytes','http','dst_bytes','count','private','dst_host_srv_count','dst_host_same_srv_rate','dst_host_same_src_port_rate','hot']
#col_list=['class','ecr_i','src_bytes','http','dst_bytes','count']


#10 feature set
#col_list=['class','src_bytes', 'dst_bytes','hot','count','dst_host_srv_count','dst_host_same_srv_rate','dst_host_same_src_port_rate','ecr_i','http','private']

#8 feature set
#col_list=['class','src_bytes', 'dst_bytes','hot','count','dst_host_same_srv_rate','ecr_i','http','private']

#6 feature set
#col_list=['class','src_bytes', 'dst_bytes','hot','count','ecr_i','http']

#4 feature set
col_list=['class','src_bytes', 'dst_bytes','ecr_i','http']

#3 feature set
#col_list=['class','src_bytes', 'ecr_i','http']

#col_list=['class','ecr_i','http','src_bytes','hot','dst_bytes','dst_host_same_srv_rate']
alldata2=alldata[col_list]
traindata=alldata2.ix[0:trainlen-1,:]
testdata=alldata2.ix[trainlen:alldata2.shape[0],:]


#X = traindata.iloc[:,1:42]
#Y = traindata.iloc[:,0]
#C = testdata.iloc[:,0]
#T = testdata.iloc[:,1:42]

X = traindata.drop('class', axis=1)
Y = traindata[['class']]
T = testdata.drop('class',axis=1)
C = testdata[['class']]

scaler = Normalizer().fit(X)
trainX = scaler.transform(X)
# summarize transformed data
np.set_printoptions(precision=3)
#print(trainX[0:5,:])

scaler = Normalizer().fit(T)
testT = scaler.transform(T)
# summarize transformed data
np.set_printoptions(precision=3)
#print(testT[0:5,:])

print("X info")
X.info()
print("Y info")
Y.info()
print("T info")
T.info()
print("C info")
C.info()
y_train = np.array(Y)
y_test = np.array(C)


# reshape input to be [samples, time steps, features]
X_train = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
X_test = np.reshape(testT, (testT.shape[0], 1, testT.shape[1]))


print(X_train.shape)
print(X_test.shape)
print(X.columns)
print(T.columns)

batch_size = 32

# 1. define the network
#model = Sequential()
#model.add(LSTM(4,input_dim=(1,122)))  # try using a GRU instead, for fun



# configure network
print("TIME BEGIN:",datetime.datetime.now())
n_epoch = 100
n_neurons = 80
# design network
model = Sequential()





model.add(LSTM(n_neurons,input_shape=(1,4)))
model.add(Dropout(0.1))
model.add(Dense(1))
model.add(Activation('sigmoid'))
#print(model.get_config())


#model = Sequential([
#    LSTM(4,input_shape=(1,122)),
#    Dense(len(class_names), activation='softmax')
#])

#model.compile(
#    optimizer='adam',
#    loss='sparse_categorical_crossentropy',
#    metrics=['accuracy'])


# try using different optimizers and different optimizer configs
sgd = optimizers.SGD(lr=0.5)
model.compile(loss='binary_crossentropy',optimizer=sgd,metrics=['accuracy'])
#model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

#checkpointer = callbacks.ModelCheckpoint(filepath="kddresults/lstm1layer/checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='val_acc',mode='max')
csv_logger = CSVLogger('4featuresetlog.csv',separator=',', append=False)
#model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=1000, validation_data=(X_test, y_test),callbacks=[checkpointer,csv_logger])
model.fit(X_train, y_train, batch_size=batch_size, epochs=45, validation_data=(X_test, y_test),callbacks=[csv_logger])

#model.save("kddresults/lstm1layer/fullmodel/lstm1layer_model.hdf5")
# Save the model
model.save('4featureModel.h5')

loss, accuracy = model.evaluate(X_test, y_test)
print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy*100))
y_pred = model.predict_classes(X_test)
print(classification_report(y_test, y_pred))
#np.savetxt('kddresults/lstm1layer/lstm1predicted.txt', y_pred, fmt='%01d')

print("TIME END:",datetime.datetime.now())






tn, fp, fn, tp = confusion_matrix(y_test,y_pred).ravel()
print(tn)
print(fp)
print(fn)
print(tp)
confusion_matrix(y_test,y_pred)

/home/dsu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/home/dsu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
W1001 13:44:23.675068 140611690805056 deprecation_wrapper.py:119] From /home/dsu/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:68: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1001 13:44:23.677129 140611690805056 deprecation_wrapper.py:119] From /home/dsu/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_

X info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125973 entries, 0 to 125972
Data columns (total 4 columns):
src_bytes    125973 non-null int64
dst_bytes    125973 non-null int64
ecr_i        125973 non-null uint8
http         125973 non-null uint8
dtypes: int64(2), uint8(2)
memory usage: 2.2 MB
Y info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125973 entries, 0 to 125972
Data columns (total 1 columns):
class    125973 non-null int64
dtypes: int64(1)
memory usage: 984.2 KB
T info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22543 entries, 125973 to 148515
Data columns (total 4 columns):
src_bytes    22543 non-null int64
dst_bytes    22543 non-null int64
ecr_i        22543 non-null uint8
http         22543 non-null uint8
dtypes: int64(2), uint8(2)
memory usage: 396.3 KB
C info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22543 entries, 125973 to 148515
Data columns (total 1 columns):
class    22543 non-null int64
dtypes: int64(1)
memory usage: 176.2 KB
(125973, 

W1001 13:44:23.865136 140611690805056 deprecation_wrapper.py:119] From /home/dsu/anaconda3/lib/python3.7/site-packages/keras/optimizers.py:744: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1001 13:44:23.869014 140611690805056 deprecation_wrapper.py:119] From /home/dsu/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3066: The name tf.log is deprecated. Please use tf.math.log instead.

W1001 13:44:23.872488 140611690805056 deprecation.py:323] From /home/dsu/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 125973 samples, validate on 22543 samples
Epoch 1/45
125973/125973 [==============================] - 38s 303us/step - loss: 0.2014 - acc: 0.9015 - val_loss: 0.8612 - val_acc: 0.7684
Epoch 2/45
125973/125973 [==============================] - 35s 280us/step - loss: 0.1890 - acc: 0.9065 - val_loss: 0.9061 - val_acc: 0.7691
Epoch 3/45
125973/125973 [==============================] - 35s 277us/step - loss: 0.1884 - acc: 0.9071 - val_loss: 0.9073 - val_acc: 0.6985
Epoch 4/45
125973/125973 [==============================] - 35s 280us/step - loss: 0.1881 - acc: 0.9068 - val_loss: 0.8373 - val_acc: 0.7683
Epoch 5/45
125973/125973 [==============================] - 35s 280us/step - loss: 0.1879 - acc: 0.9079 - val_loss: 0.8176 - val_acc: 0.7682
Epoch 6/45
125973/125973 [==============================] - 35s 278us/step - loss: 0.1879 - acc: 0.9078 - val_loss: 0.8674 - val_acc: 0.7686
Epoch 7/45
125973/125973 [==============================] - 33s 261us/step - loss: 0.1876 - acc: 0.9081

array([[8354, 4479],
       [ 741, 8969]])

In [6]:
metrics.f1_score(y_test, y_pred, labels=np.unique(y_pred))

0.7745919336730287

# 6 feature set

In [5]:
alldata.head()
#col_list=['dst_host_diff_srv_rate','dst_host_serror_rate','serror_rate','dst_host_same_src_port_rate','S0','dst_host_srv_serror_rate','class']
#col_list=['class','ecr_i','src_bytes','http','dst_bytes','count','private','dst_host_srv_count','dst_host_same_srv_rate','dst_host_same_src_port_rate','hot','dst_host_srv_serror_rate','is_guest_login','icmp','REJ','dst_host_serror_rate','dst_host_srv_diff_host_rate','tcp','urp_i','dst_host_rerror_rate','finger']
#col_list=['class','ecr_i','src_bytes','http','dst_bytes','count','private','dst_host_srv_count','dst_host_same_srv_rate','dst_host_same_src_port_rate','hot']
#col_list=['class','ecr_i','src_bytes','http','dst_bytes','count']


#10 feature set
#col_list=['class','src_bytes', 'dst_bytes','hot','count','dst_host_srv_count','dst_host_same_srv_rate','dst_host_same_src_port_rate','ecr_i','http','private']

#8 feature set
#col_list=['class','src_bytes', 'dst_bytes','hot','count','dst_host_same_srv_rate','ecr_i','http','private']

#6 feature set
col_list=['class','src_bytes', 'dst_bytes','hot','count','ecr_i','http']

#4 feature set
#col_list=['class','src_bytes', 'dst_bytes','ecr_i','http']

#3 feature set
#col_list=['class','src_bytes', 'ecr_i','http']

#col_list=['class','ecr_i','http','src_bytes','hot','dst_bytes','dst_host_same_srv_rate']
alldata2=alldata[col_list]
traindata=alldata2.ix[0:trainlen-1,:]
testdata=alldata2.ix[trainlen:alldata2.shape[0],:]


#X = traindata.iloc[:,1:42]
#Y = traindata.iloc[:,0]
#C = testdata.iloc[:,0]
#T = testdata.iloc[:,1:42]

X = traindata.drop('class', axis=1)
Y = traindata[['class']]
T = testdata.drop('class',axis=1)
C = testdata[['class']]

scaler = Normalizer().fit(X)
trainX = scaler.transform(X)
# summarize transformed data
np.set_printoptions(precision=3)
#print(trainX[0:5,:])

scaler = Normalizer().fit(T)
testT = scaler.transform(T)
# summarize transformed data
np.set_printoptions(precision=3)
#print(testT[0:5,:])

print("X info")
X.info()
print("Y info")
Y.info()
print("T info")
T.info()
print("C info")
C.info()
y_train = np.array(Y)
y_test = np.array(C)


# reshape input to be [samples, time steps, features]
X_train = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
X_test = np.reshape(testT, (testT.shape[0], 1, testT.shape[1]))


print(X_train.shape)
print(X_test.shape)
print(X.columns)
print(T.columns)

batch_size = 32

# 1. define the network
#model = Sequential()
#model.add(LSTM(4,input_dim=(1,122)))  # try using a GRU instead, for fun



# configure network
print("TIME BEGIN:",datetime.datetime.now())
n_epoch = 100
n_neurons = 80
# design network
model = Sequential()





model.add(LSTM(n_neurons,input_shape=(1,6)))
model.add(Dropout(0.1))
model.add(Dense(1))
model.add(Activation('sigmoid'))
#print(model.get_config())


#model = Sequential([
#    LSTM(4,input_shape=(1,122)),
#    Dense(len(class_names), activation='softmax')
#])

#model.compile(
#    optimizer='adam',
#    loss='sparse_categorical_crossentropy',
#    metrics=['accuracy'])


# try using different optimizers and different optimizer configs
sgd = optimizers.SGD(lr=0.5)
model.compile(loss='binary_crossentropy',optimizer=sgd,metrics=['accuracy'])
#model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

#checkpointer = callbacks.ModelCheckpoint(filepath="kddresults/lstm1layer/checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='val_acc',mode='max')
csv_logger = CSVLogger('6featuresetlog.csv',separator=',', append=False)
#model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=1000, validation_data=(X_test, y_test),callbacks=[checkpointer,csv_logger])
model.fit(X_train, y_train, batch_size=batch_size, epochs=45, validation_data=(X_test, y_test),callbacks=[csv_logger])

#model.save("kddresults/lstm1layer/fullmodel/lstm1layer_model.hdf5")
# Save the model
model.save('6featureModel.h5')

loss, accuracy = model.evaluate(X_test, y_test)
print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy*100))
y_pred = model.predict_classes(X_test)
print(classification_report(y_test, y_pred))
#np.savetxt('kddresults/lstm1layer/lstm1predicted.txt', y_pred, fmt='%01d')

print("TIME END:",datetime.datetime.now())






tn, fp, fn, tp = confusion_matrix(y_test,y_pred).ravel()
print(tn)
print(fp)
print(fn)
print(tp)
confusion_matrix(y_test,y_pred)

/home/dsu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/home/dsu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
W1001 14:23:16.453169 140510210565952 deprecation_wrapper.py:119] From /home/dsu/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:68: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1001 14:23:16.455965 140510210565952 deprecation_wrapper.py:119] From /home/dsu/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_

X info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125973 entries, 0 to 125972
Data columns (total 6 columns):
src_bytes    125973 non-null int64
dst_bytes    125973 non-null int64
hot          125973 non-null int64
count        125973 non-null int64
ecr_i        125973 non-null uint8
http         125973 non-null uint8
dtypes: int64(4), uint8(2)
memory usage: 4.1 MB
Y info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125973 entries, 0 to 125972
Data columns (total 1 columns):
class    125973 non-null int64
dtypes: int64(1)
memory usage: 984.2 KB
T info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22543 entries, 125973 to 148515
Data columns (total 6 columns):
src_bytes    22543 non-null int64
dst_bytes    22543 non-null int64
hot          22543 non-null int64
count        22543 non-null int64
ecr_i        22543 non-null uint8
http         22543 non-null uint8
dtypes: int64(4), uint8(2)
memory usage: 748.6 KB
C info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22543

W1001 14:23:16.612241 140510210565952 deprecation.py:506] From /home/dsu/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3135: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W1001 14:23:16.643925 140510210565952 deprecation_wrapper.py:119] From /home/dsu/anaconda3/lib/python3.7/site-packages/keras/optimizers.py:744: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1001 14:23:16.649744 140510210565952 deprecation_wrapper.py:119] From /home/dsu/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3066: The name tf.log is deprecated. Please use tf.math.log instead.

W1001 14:23:16.656026 140510210565952 deprecation.py:323] From /home/dsu/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/nn_impl.py:180:

Train on 125973 samples, validate on 22543 samples
Epoch 1/45
125973/125973 [==============================] - 51s 408us/step - loss: 0.1756 - acc: 0.9300 - val_loss: 0.8794 - val_acc: 0.7462
Epoch 2/45
125973/125973 [==============================] - 49s 387us/step - loss: 0.1530 - acc: 0.9425 - val_loss: 0.9360 - val_acc: 0.7217
Epoch 3/45
125973/125973 [==============================] - 48s 385us/step - loss: 0.1512 - acc: 0.9453 - val_loss: 0.8779 - val_acc: 0.7233
Epoch 4/45
125973/125973 [==============================] - 48s 384us/step - loss: 0.1500 - acc: 0.9469 - val_loss: 0.8521 - val_acc: 0.7450
Epoch 5/45
125973/125973 [==============================] - 48s 379us/step - loss: 0.1491 - acc: 0.9477 - val_loss: 0.8024 - val_acc: 0.7464
Epoch 6/45
125973/125973 [==============================] - 48s 378us/step - loss: 0.1482 - acc: 0.9490 - val_loss: 0.8577 - val_acc: 0.7360
Epoch 7/45
125973/125973 [==============================] - 47s 377us/step - loss: 0.1475 - acc: 0.9495

array([[7755, 5078],
       [ 423, 9287]])

In [6]:
metrics.f1_score(y_test, y_pred, labels=np.unique(y_pred))

0.7715057113187955

# 8 feature set

In [5]:
alldata.head()
#col_list=['dst_host_diff_srv_rate','dst_host_serror_rate','serror_rate','dst_host_same_src_port_rate','S0','dst_host_srv_serror_rate','class']
#col_list=['class','ecr_i','src_bytes','http','dst_bytes','count','private','dst_host_srv_count','dst_host_same_srv_rate','dst_host_same_src_port_rate','hot','dst_host_srv_serror_rate','is_guest_login','icmp','REJ','dst_host_serror_rate','dst_host_srv_diff_host_rate','tcp','urp_i','dst_host_rerror_rate','finger']
#col_list=['class','ecr_i','src_bytes','http','dst_bytes','count','private','dst_host_srv_count','dst_host_same_srv_rate','dst_host_same_src_port_rate','hot']
#col_list=['class','ecr_i','src_bytes','http','dst_bytes','count']


#10 feature set
#col_list=['class','src_bytes', 'dst_bytes','hot','count','dst_host_srv_count','dst_host_same_srv_rate','dst_host_same_src_port_rate','ecr_i','http','private']

#8 feature set
col_list=['class','src_bytes', 'dst_bytes','hot','count','dst_host_same_srv_rate','ecr_i','http','private']

#6 feature set
#col_list=['class','src_bytes', 'dst_bytes','hot','count','ecr_i','http']

#4 feature set
#col_list=['class','src_bytes', 'dst_bytes','ecr_i','http']

#3 feature set
#col_list=['class','src_bytes', 'ecr_i','http']

#col_list=['class','ecr_i','http','src_bytes','hot','dst_bytes','dst_host_same_srv_rate']
alldata2=alldata[col_list]
traindata=alldata2.ix[0:trainlen-1,:]
testdata=alldata2.ix[trainlen:alldata2.shape[0],:]


#X = traindata.iloc[:,1:42]
#Y = traindata.iloc[:,0]
#C = testdata.iloc[:,0]
#T = testdata.iloc[:,1:42]

X = traindata.drop('class', axis=1)
Y = traindata[['class']]
T = testdata.drop('class',axis=1)
C = testdata[['class']]

scaler = Normalizer().fit(X)
trainX = scaler.transform(X)
# summarize transformed data
np.set_printoptions(precision=3)
#print(trainX[0:5,:])

scaler = Normalizer().fit(T)
testT = scaler.transform(T)
# summarize transformed data
np.set_printoptions(precision=3)
#print(testT[0:5,:])

print("X info")
X.info()
print("Y info")
Y.info()
print("T info")
T.info()
print("C info")
C.info()
y_train = np.array(Y)
y_test = np.array(C)


# reshape input to be [samples, time steps, features]
X_train = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
X_test = np.reshape(testT, (testT.shape[0], 1, testT.shape[1]))


print(X_train.shape)
print(X_test.shape)
print(X.columns)
print(T.columns)

batch_size = 32

# 1. define the network
#model = Sequential()
#model.add(LSTM(4,input_dim=(1,122)))  # try using a GRU instead, for fun



# configure network
print("TIME BEGIN:",datetime.datetime.now())
n_epoch = 100
n_neurons = 80
# design network
model = Sequential()





model.add(LSTM(n_neurons,input_shape=(1,8)))
model.add(Dropout(0.1))
model.add(Dense(1))
model.add(Activation('sigmoid'))
#print(model.get_config())


#model = Sequential([
#    LSTM(4,input_shape=(1,122)),
#    Dense(len(class_names), activation='softmax')
#])

#model.compile(
#    optimizer='adam',
#    loss='sparse_categorical_crossentropy',
#    metrics=['accuracy'])


# try using different optimizers and different optimizer configs
sgd = optimizers.SGD(lr=0.5)
model.compile(loss='binary_crossentropy',optimizer=sgd,metrics=['accuracy'])
#model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

#checkpointer = callbacks.ModelCheckpoint(filepath="kddresults/lstm1layer/checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='val_acc',mode='max')
csv_logger = CSVLogger('8featuresetlog.csv',separator=',', append=False)
#model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=1000, validation_data=(X_test, y_test),callbacks=[checkpointer,csv_logger])
model.fit(X_train, y_train, batch_size=batch_size, epochs=45, validation_data=(X_test, y_test),callbacks=[csv_logger])

#model.save("kddresults/lstm1layer/fullmodel/lstm1layer_model.hdf5")
# Save the model
model.save('8featureModel.h5')

loss, accuracy = model.evaluate(X_test, y_test)
print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy*100))
y_pred = model.predict_classes(X_test)
print(classification_report(y_test, y_pred))
#np.savetxt('kddresults/lstm1layer/lstm1predicted.txt', y_pred, fmt='%01d')

print("TIME END:",datetime.datetime.now())






tn, fp, fn, tp = confusion_matrix(y_test,y_pred).ravel()
print(tn)
print(fp)
print(fn)
print(tp)
confusion_matrix(y_test,y_pred)

/home/dsu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/home/dsu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
W1001 15:47:23.960289 140350102923072 deprecation_wrapper.py:119] From /home/dsu/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:68: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1001 15:47:23.964316 140350102923072 deprecation_wrapper.py:119] From /home/dsu/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_

X info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125973 entries, 0 to 125972
Data columns (total 8 columns):
src_bytes                 125973 non-null int64
dst_bytes                 125973 non-null int64
hot                       125973 non-null int64
count                     125973 non-null int64
dst_host_same_srv_rate    125973 non-null float64
ecr_i                     125973 non-null uint8
http                      125973 non-null uint8
private                   125973 non-null uint8
dtypes: float64(1), int64(4), uint8(3)
memory usage: 5.2 MB
Y info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125973 entries, 0 to 125972
Data columns (total 1 columns):
class    125973 non-null int64
dtypes: int64(1)
memory usage: 984.2 KB
T info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22543 entries, 125973 to 148515
Data columns (total 8 columns):
src_bytes                 22543 non-null int64
dst_bytes                 22543 non-null int64
hot                       22543 no

W1001 15:47:24.143772 140350102923072 deprecation_wrapper.py:119] From /home/dsu/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:126: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W1001 15:47:24.149924 140350102923072 deprecation.py:506] From /home/dsu/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3135: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W1001 15:47:24.185242 140350102923072 deprecation_wrapper.py:119] From /home/dsu/anaconda3/lib/python3.7/site-packages/keras/optimizers.py:744: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1001 15:47:24.189196 140350102923072 deprecation_wrapper.py:119] From /home/dsu/anaconda3/lib/python3

Train on 125973 samples, validate on 22543 samples
Epoch 1/45
125973/125973 [==============================] - 50s 397us/step - loss: 0.1746 - acc: 0.9247 - val_loss: 0.8836 - val_acc: 0.7479
Epoch 2/45
125973/125973 [==============================] - 47s 373us/step - loss: 0.1470 - acc: 0.9454 - val_loss: 0.9539 - val_acc: 0.7208
Epoch 3/45
125973/125973 [==============================] - 47s 377us/step - loss: 0.1366 - acc: 0.9564 - val_loss: 0.8962 - val_acc: 0.7320
Epoch 4/45
125973/125973 [==============================] - 47s 372us/step - loss: 0.1277 - acc: 0.9595 - val_loss: 0.8905 - val_acc: 0.7326
Epoch 5/45
125973/125973 [==============================] - 47s 375us/step - loss: 0.1226 - acc: 0.9603 - val_loss: 0.8310 - val_acc: 0.7453
Epoch 6/45
125973/125973 [==============================] - 47s 373us/step - loss: 0.1200 - acc: 0.9610 - val_loss: 0.9401 - val_acc: 0.7350
Epoch 7/45
125973/125973 [==============================] - 47s 375us/step - loss: 0.1188 - acc: 0.9614

array([[7267, 5566],
       [ 335, 9375]])

In [6]:
metrics.f1_score(y_test, y_pred, labels=np.unique(y_pred))

0.7606182304977485